In [1]:
import numpy as np
import pandas as pd
import dateutil as dtutil
import pytz
import six

In [2]:
ratings = pd.read_csv('ratings.csv', encoding='latin-1', sep=',|:')
ratings.head()

/tmp/ipykernel_17140/3026495763.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ratings = pd.read_csv('ratings.csv', encoding='latin-1', sep=',|:')


,UserId,ItemId,Rating
0,4baf0ac888,8b05db84f2,3
1,f3364fd7e6,8b05db84f2,2
2,72a8982204,8b05db84f2,5
3,d0171f7a9d,e40cec3cbe,5
4,e6dad08499,e40cec3cbe,5


In [4]:
def get_user_item_mat(df):
    num_users = df.iloc[:, 0].nunique()
    num_items = df.iloc[:, 1].nunique()
    # user_item_mat = np.empty((num_users, num_items))
    # user_item_mat.fill(np.nan)

    user_item_mat = np.full((num_users, num_items), fill_value=np.nan)

    user_id_lookup = dict(zip(df.iloc[:, 0].unique(), range(num_users)))
    item_id_lookup = dict(zip(df.iloc[:, 1].unique(), range(num_items)))

    # users_keys = user_id_lookup.keys()
    # items_keys = item_id_lookup.keys()

    for idx, row in df.iterrows():
        user_item_mat[user_id_lookup[row[0]], item_id_lookup[row[1]]] = row[2]
    # return user_item_mat, users_keys, items_keys
    return user_item_mat

In [5]:
def funkSVD(ratings_mat, latent_features=4, learning_rate=0.0001, iters=100):
    num_users = ratings_mat.shape[0]
    num_movies = ratings_mat.shape[1]
    num_ratings = np.count_nonzero(~np.isnan(ratings_mat))

    users_latent = np.random.random_sample((num_users, latent_features))
    movies_latent = np.random.random_sample((latent_features, num_movies))

    for iter_num in range(iters):
        cum_sse = 0
        for row in range(num_users):
            for col in range(num_movies):
                if ratings_mat[row, col] > 0:  # rating exists
                    predicted_value = np.dot(
                        users_latent[row, :], movies_latent[:, col]
                    )
                    error = ratings_mat[row, col] - predicted_value
                    cum_sse += np.power(error, 2)

                    weight = learning_rate * 2 * (error)
                    users_latent[row, :] += weight * movies_latent[:, col]
                    movies_latent[:, col] += weight * users_latent[row, :]
        print(f"Iteration: {iter_num + 1}   MSE: {cum_sse/num_ratings:.6f}")

    return users_latent, movies_latent


In [6]:
def predict_rating(df, user_id, movie_id, user_mat, movie_mat):
    """TO DO"""

    user_id_vals = df.loc[:, "UserId"].unique()
    movie_id_vals = df.loc[:, "MovieId"].unique()

    # try:
    user_idx = np.where(user_id_vals == user_id)[0][0]
    movie_idx = np.where(movie_id_vals == movie_id)[0][0]
    return np.dot(user_mat[user_idx, :], movie_mat[:, movie_idx])
    # except IndexError:
    #     print("Either the user or movie is missing from the our database")
    #     return None


In [7]:
rating_mat = get_user_item_mat(ratings)
ulat, ilat = funkSVD(rating_mat, latent_features=10, learning_rate=0.0001, iters=10)


MemoryError: Unable to allocate 168. GiB for an array with shape (269268, 83767) and data type float64

In [ ]:
outfile = open('result.csv', 'w')
targets = pd.read_csv('ratings.csv', encoding='latin-1', sep=':')
for index, row in targets.iterrows():
    predicted_value = predict_rating(row['UserId'], row['ItemId'], ulat, ilat, rating_mat)
    print("{user}:{item},{value}".format(row['UserId'], row['ItemId'], predicted_value),
    file=outfile)